In [ ]:
# CURRENTLY WORKING IN THIS ONE
import os
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from PIL import Image
import numpy as np

In [ ]:
image_size = 64
INPUT_DATA_PATH = 'test_input.npy'
OUTPUT_DATA_PATH = 'test_output.npy'
X_train = np.load(INPUT_DATA_PATH)
Y_train = np.load(OUTPUT_DATA_PATH)

In [ ]:
class LinesDataset():
        def __init__(self, lines, labels):
                self.lines = lines
                self.labels = labels
        def __len__(self):
                return len(self.lines)
        def __getitem__(self, idx):
                line = self.lines[idx]
                label = self.labels[idx]
                sample = {"Line": line, "Label": label}
                return sample

# def show_batch(ds):
        
#         ds_iter = iter(ds)
        
#         for i in range (len(ds)):
#                 samp = next(ds_iter)
#                 line = samp['Line']
#                 label = samp['Label']
#                 plt.imshow(line)
#                 plt.axis('off')
#                 plt.show()

training_dataset = LinesDataset(X_train, Y_train)
# show_batch(training_dataset)

# create DataLoader object of DataSet object
bat_size = 2
DL_DS = DataLoader(training_dataset, batch_size=bat_size, shuffle=True)

# we need a data loader for the test data as well

# # loop through each batch in the DataLoader object
# for (idx, batch) in enumerate(DL_DS):
#     print(idx, 'Text data: ', batch, '\n')


In [ ]:
# create a CNN
class NeuralNetwork(nn.Module):
    def __init__(self, input_channels, input_size):
        super(NeuralNetwork, self).__init__()

        self.cnn_stack = nn.Sequential(
            nn.Conv2d(in_channels=input_channels, out_channels=1, kernel_size=(9,9), stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None),
            # convolutional layers + activation functions and pooling layers in between
            # nn.Conv2d(in_channels=input_channels, out_channels=1, kernel_size=(9,9), stride=(1,1))
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        )

    def forward(self, x):
        logits = self.cnn_stack(x)
        return logits

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        # print(f"predicted: {pred}")
        # print(f"actual: {y}")
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # print(f"loss: {loss:>7f}")

        if batch % 50 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
model = NeuralNetwork(3, (576, 576))
learning_rate = 0.001
batch_size = 64
epochs = 5
# Initialize the loss function
# loss_fn = nn.MSELoss()
loss_fn = nn.CrossEntropyLoss() # can change this to another loss function
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(DL_DS, model, loss_fn, optimizer)
    # test_loop(test_dataloader, model, loss_fn)
    
print("Done!")